## 使用 Gluon 实现RNN, 创作歌词
### 1. 需要 想想怎么在 Colab 解决 代码复用的问题
- mount google drive
- clone the repo or change to the repo
- pull code
- open the ipynb file 
- run it

### 2. 





In [ ]:
# 准备 Google Colab 环境: 在Runtime中选择 GPU
# 拉取数据集
pwd = !pwd
if 'dive_into_deep_learning' not in pwd[0]:
    ! git clone https://github.com/chibinjiang/dive_into_deep_learning.git
    # 进入到和开发环境相似的工作目录
%cd /content/dive_into_deep_learning/
# 安装依赖
! pip install mxnet-cu101mkl

In [9]:
import re
import math
import time
import zipfile
import traceback
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, rnn

In [43]:
# 加载数据集
with zipfile.ZipFile('DataResources/Chapter_6/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
corpus_chars = re.sub(r'\s+', ' ', corpus_chars)
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
corpus_indices = [char_to_idx[char] for char in corpus_chars]

In [28]:
# 定义 RNN 模型
def gen_net(num_hiddens, batch_size):
    _rnn = rnn.RNN(num_hiddens)
    _rnn.initialize()
    _rnn.begin_state(batch_size=batch_size)
    return _rnn
    
class RNN(nn.Block):
    def __init__(self, num_hiddens, batch_size, vocab_size, **kwargs):
        super(RNN, self).__init__(**kwargs)
        self.rnn = gen_net(num_hiddens, batch_size)
        self.vocab_size = vocab_size
        self.dense = nn.Dense(vocab_size)  # 为什么需要一个全连接层
    
    def forward(self, inputs, state):
        # 将输入转置成(num_steps, batch_size)后获取one-hot向量表示
        X = nd.one_hot(inputs.T, self.vocab_size)
        Y, state = self.rnn(X, state)
        # 全连接层会首先将Y的形状变成(num_steps * batch_size, num_hiddens)，它的输出
        # 形状为(num_steps * batch_size, vocab_size)
        output = self.dense(Y.reshape((-1, Y.shape[-1])))
        return output, state
    
    def begin_state(self, *args, **kwargs):
        return self.rnn.begin_state(*args, **kwargs)

In [29]:
temp_net = gen_net(100, 2)

In [30]:
init_state = temp_net.begin_state(batch_size=2)
X = nd.random.uniform(shape=(30, 2, vocab_size))
Y, state_new = temp_net(X, init_state)
(Y.shape, state_new)

((30, 2, 100), [
  [[[1.5278673  0.         1.0445228  0.         0.         1.6484566
     1.8354766  0.         0.         0.         1.1338558  0.
     0.42351055 0.         0.         0.         0.         1.3539939
     0.31653637 0.         0.         0.4922253  0.         1.4564292
     1.1331655  0.08769092 0.2364367  0.         0.5829359  3.5316272
     0.         0.         0.08696572 0.         1.5529106  0.6949022
     0.         0.57017994 0.         0.         0.19693512 0.
     0.         0.         0.08501232 1.3938953  0.         0.5927223
     0.         0.         0.         1.7381938  0.7154095  0.3993885
     0.         0.         0.5789365  0.16283596 0.         0.
     4.122617   0.         0.         0.7438283  0.         0.9080451
     1.1469073  0.66469157 0.00958797 0.         0.7123618  0.
     0.67446774 0.         0.08351076 2.5886292  0.         1.9853334
     1.7640082  0.         0.15815529 0.22243357 0.         0.
     0.7966601  0.         1.4879034  

In [31]:
def predict_rnn_gluon(prefix, num_chars, model, vocab_size, ctx, idx_to_char, char_to_idx):
    """
    预测prefix 之后的歌词
    """
    state = model.begin_state(batch_size=1, ctx=ctx)  # 使用model的成员函数来初始化隐藏状态
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        X = nd.array([output[-1]], ctx=ctx).reshape((1, 1))
        (Y, state) = model(X, state)  # 前向计算不需要传入模型参数
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

In [34]:
def try_gpu(gpu_number=0):
    """
        Return gpu(i) if exists, otherwise return cpu().
    """
#     import traceback
    try:
        _ = mx.nd.array([1, 2, 3], ctx=mx.gpu(gpu_number))
        print("Try GPU: {}".format(gpu_number))
    except mx.MXNetError:
#         traceback.print_exc()
        print("Try CPU: {}".format(gpu_number))
        return mx.cpu()
    return mx.gpu(gpu_number)

In [35]:
ctx = try_gpu()
model = RNN(256, 2, vocab_size)
model.initialize(force_reinit=True, ctx=ctx)
predict_rnn_gluon('分开', 10, model, vocab_size, ctx, idx_to_char, char_to_idx)

Try CPU: 0


'分开贪期隧砍術演墨币样阵'

In [38]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx=None):
    """
    相邻采样: 相邻 epoch 的 batch_size 样本是相邻的
    Sample mini-batches in a consecutive order from sequential data.
    """
    corpus_indices = nd.array(corpus_indices, ctx=ctx)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0 : batch_size * batch_len].reshape((
        batch_size, batch_len))  # 只要 前面的batch_size * batch_len 个 
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i : i + num_steps]
        Y = indices[:, i + 1 : i + num_steps + 1]
        yield X, Y

In [39]:
# 剪裁梯度
def grad_clipping(params, theta, ctx):
    norm = nd.array([0], ctx)
    for param in params:
        norm += (param.grad ** 2).sum()
    norm = norm.sqrt().asscalar()
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [40]:
def train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    perplexity_hist = list()
    loss = gloss.SoftmaxCrossEntropyLoss()
    model.initialize(ctx=ctx, force_reinit=True, init=init.Normal(0.01))
    trainer = gluon.Trainer(model.collect_params(), 'sgd',
                            {'learning_rate': lr, 'momentum': 0, 'wd': 0})

    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_consecutive(
            corpus_indices, batch_size, num_steps, ctx)
        state = model.begin_state(batch_size=batch_size, ctx=ctx)
        for X, Y in data_iter:
            for s in state:
                s.detach()
            with autograd.record():
                (output, state) = model(X, state)
                y = Y.T.reshape((-1,))
                l = loss(output, y).mean()
            l.backward()
            # 梯度裁剪
            params = [p.data() for p in model.collect_params().values()]
            grad_clipping(params, clipping_theta, ctx)
            trainer.step(1)  # 因为已经误差取过均值，梯度不用再做平均
            l_sum += l.asscalar() * y.size
            n += y.size
        perplexity = math.exp(l_sum / n)
        perplexity_hist.append(perplexity)
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_gluon(prefix, pred_len, model, vocab_size, ctx, idx_to_char, char_to_idx))
    return perplexity_hist

In [45]:
num_hiddens, num_steps = 256, 35
num_epochs, batch_size, lr, clipping_theta = 300, 32, 100, 0.01
pred_period, pred_len, prefixes = 10, 50, ['分开', '不分开', '我静静地']
perplexities = train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 10, perplexity 293.331648, time 8.80 sec
 - 分开 我们 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我
 - 不分开 我们的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 
 - 我静静地 我们 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我
epoch 20, perplexity 166.335066, time 8.72 sec
 - 分开 我们的感 在一种 你的世界 在空的 我有不着 你的世界 你的爱界 你的爱界 你的爱界 你的爱界 你
 - 不分开 我们不起 你的世界 在空的 我有不着 你的世界 你的爱界 你的爱界 你的爱界 你的爱界 你的爱界 
 - 我静静地 你 不起 你的世界 你的爱界 你的爱界 你的爱界 你的爱界 你的爱界 你的爱界 你的爱界 你的爱界
epoch 30, perplexity 93.325684, time 8.76 sec
 - 分开 你说不起 让我们乘着阳光 我用度坚尝 当敌 是谁 我用 不到 不了麻烦了 不用麻烦了 不用麻烦了 
 - 不分开 我要不到 你没有一人 我们势如龙 当敌人是空 我目势如龙 当敌人是空 我目势如龙 当敌人是空 我目
 - 我静静地 她不用 让我用 你不需 不要 不了 不需 不了 不需 不了 不需 不了 不需 不了 不需 不了 不
epoch 40, perplexity 55.906913, time 8.77 sec
 - 分开 我们的时间 当目标的空寸 原来的很 你让我们热 每一天的 漫 一直 在谁在 的灵魂 我们 有你的 
 - 不分开 我知道如穷 当敌人是空 我左右如龙 当敌人尽空 不用法红豆 不用麻烦了 不用麻烦了 不用麻烦了 不
 - 我静静地 你就是虽 在飞了 的灵情 单纯一停 等一直人 我等一起热 让我们乘着阳光 看着远方 去支耿心 汹不
epoch 50, perplexity 37.384783, time 9.95 sec
 - 分开 我们的时情 全在一遍秀 让我们 半兽人 的灵魂 我们无整恼 我目光 追绪 天 这些 Y飘 H硬 得
 - 不分开 她手的影匙 我们中个世色 未哼哈壁 快使用

KeyboardInterrupt: 

In [ ]:
num_hiddens, num_steps = 256, 35
num_epochs, batch_size, lr, clipping_theta = 300, 32, 100, 0.01
pred_period, pred_len, prefixes = 10, 50, ['分开', '不分开', '我静静地']
perplexities = train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

In [ ]:
perplexities